# Configure

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path
import sys
import glob
from os.path import join
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio 
import rasterio.plot
import rasterio.mask
import json
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Rectangle
import matplotlib.colors as mpc
from matplotlib import colormaps, cm
from matplotlib.collections import PatchCollection

from util.files import *
from util.const import *
from util.ddfs import *



# Load ensemble and default estimates

In [ ]:
ens_df = pd.read_parquet(join(FO, 'ensemble.pqt'))
def_df = pd.read_parquet(join(FO, 'benchmark_loss.pqt'))

In [ ]:
# Also load in damage functions
# Load DDFs
naccs_ddfs = pd.read_parquet(join(VULN_DIR_I, 'physical', 'naccs_ddfs.pqt'))
hazus_ddfs = pd.read_parquet(join(VULN_DIR_I, 'physical', 'hazus_ddfs.pqt'))
hazus_nounc = pd.read_parquet(join(VULN_DIR_I, 'physical', 'hazus_ddfs_nounc.pqt'))

# Load helper dictionaries
with open(join(VULN_DIR_I, 'physical', 'hazus.json'), 'r') as fp:
    HAZUS_MAX_DICT = json.load(fp)

with open(join(VULN_DIR_I, 'physical', 'hazus_nounc.json'), 'r') as fp:
    HAZUS_MAX_NOUNC_DICT = json.load(fp)

with open(join(VULN_DIR_I, 'physical', 'naccs.json'), 'r') as fp:
    NACCS_MAX_DICT = json.load(fp)

# Visualize the ensemble

## Compare aggregate EAL

In [ ]:
# Plot of the pdf of summed EAL within each SOW against the
# single value from default method
ens_sums = ens_df.groupby(['sow_ind'])['eal'].sum()/1e6
fig, ax = plt.subplots(figsize=(10, 6),
                       dpi=600)
ens_sums.plot(kind='hist', ax=ax, color='blue', bins=50, alpha=.2,
              label='histogram of EAL sums\nacross ensemble members')
ax.vlines(def_df['eal'].sum()/1e6, 0, 690, color='red', label='HAZUS Default EAL')
ax.vlines(ens_sums.mean(), 0, 632, color='blue', label='Mean EAL across ensemble')
ax.vlines(ens_sums.quantile(.95), 0, 150, color='darkblue', label='95%ile EAL across ensemble')
ax.set_xlabel('Total Expected Annual Loss (EAL) Across Homes at Risk ($ Millions)', size=12)
ax.set_ylabel('Frequency Across Ensemble Members', size=12)
ax.legend(loc='upper right')


# Save fig
fig.savefig('/jumbo/keller-lab/projects/icom/nsi_unc/figures/benchmark.png',
            bbox_inches='tight',
            dpi=600)

## Compare property level EAL

In [ ]:
# Look at the proeprty with the largest variance in EAL across SOWs
# Plot the distribution of EAL for this property and plot
# what the benchmark point estimate is
max_var_id = ens_df.groupby(['fd_id'])['eal'].std().idxmax()
max_var_ens = ens_df[ens_df['fd_id'] == max_var_id]
max_var_def = def_df[def_df['fd_id'] == max_var_id]
fig, ax = plt.subplots(figsize=(10, 6))
max_var_ens['eal'].plot(kind='hist', ax=ax, color='blue', bins=50, alpha=.2,
                        label='Histogram of EAL for structure with\nlargest EAL variance across ensemble members')
ax.vlines(max_var_def['eal'], 0, 670, color='red', alpha=1, label='HAZUS Default EAL for same structure')
ax.vlines(max_var_ens['eal'].mean(), 0, 670, color='blue', label='Mean EAL across ensemble for structure')
ax.vlines(max_var_ens['eal'].quantile(.95), 0, 120, color='darkblue', label='95%ile EAL across ensemble for structure')
ax.set_xlabel('Total EAL ($)')
ax.legend(loc='upper right')

In [ ]:
# Look at the proeprty with the smallest variance in EAL across SOWs
# Plot the distribution of EAL for this property and plot
# what the benchmark point estimate is
min_var_id = ens_df.groupby(['fd_id'])['eal'].std().idxmin()
min_var_ens = ens_df[ens_df['fd_id'] == min_var_id]
min_var_def = def_df[def_df['fd_id'] == min_var_id]
fig, ax = plt.subplots(figsize=(10, 6))
min_var_ens['eal'].plot(kind='hist', ax=ax, color='blue', bins=50, alpha=.2,
                        label='Histogram of EAL for structure with\nsmallest EAL variance across ensemble members')
ax.vlines(min_var_def['eal'], 0, 670, color='red', alpha=1, label='HAZUS Default EAL for same structure')
ax.vlines(min_var_ens['eal'].mean(), 0, 670, color='blue', label='Mean EAL across ensemble for structure')
ax.vlines(min_var_ens['eal'].quantile(.95), 0, 120, color='darkblue', label='95%ile EAL across ensemble for structure')
ax.set_xlabel('Total EAL ($)')
ax.legend(loc='upper right')

In [ ]:
# Calculate the difference between mean EAL and default EAL
# Plot the distribution of these differences (the bias)
ens_means = ens_df.groupby(['fd_id'])['eal'].mean().reset_index()
mean_def_diff = pd.merge(ens_means,
                         def_df[['fd_id', 'eal']],
                         on='fd_id',
                         suffixes=['_ens', '_def'])
mean_def_diff['pct_bias'] = 100*(mean_def_diff['eal_def'] - mean_def_diff['eal_ens'])/mean_def_diff['eal_ens']
mean_def_diff['bias'] = (mean_def_diff['eal_def'] - mean_def_diff['eal_ens'])
fig, ax = plt.subplots(figsize=(14, 6), ncols=2)
mean_def_diff['pct_bias'].plot(kind='hist', ax=ax[0], color='blue', bins=50, alpha=.2,
                           label='Property-level NSI % bias\nrelative to ensemble mean EAL')
ax[0].vlines(mean_def_diff['pct_bias'].mean(), 0, 157, color='red', alpha=1, label='Mean % bias')
ax[0].vlines(mean_def_diff['pct_bias'].median(), 0, 110, color='darkblue', alpha=1, label='Median % bias')
ax[0].set_xlabel('Percent Bias (%)')
ax[0].legend(loc='upper left')

mean_def_diff['bias'].plot(kind='hist', ax=ax[1], color='blue', bins=50, alpha=.2,
                           label='Property-level NSI bias\nrelative to ensemble mean EAL')
ax[1].vlines(mean_def_diff['bias'].mean(), 0, 570, color='red', alpha=1, label='Mean bias')
ax[1].vlines(mean_def_diff['bias'].median(), 0, 570, color='darkblue', alpha=1, label='Median bias')
ax[1].set_xlabel('Bias ($)')
ax[1].legend(loc='upper left')
ax[1].set_xlim([-2000, 1500])
fig.tight_layout()

# Compare inputs with and without uncertainty

In [ ]:
# Compare ddfs w/o uncertainty to those with uncertainty
# Let's do 1SWB, the most common occtype in our case study
# according to the no uncertainty data
ddf_id = '2SWB'
hazus_id = '2SWB_A'
naccs_plot = naccs_ddfs[naccs_ddfs['ddf_id'] == ddf_id]
hazus_plot = hazus_ddfs[hazus_ddfs['ddf_id'] == hazus_id]
hazus_nounc_plot = hazus_nounc[hazus_nounc['ddf_id'] == hazus_id]

In [ ]:
# Add low/high to hazus_plot
hazus_plot[['low', 'high']] = pd.DataFrame(hazus_plot['params'].tolist(),
                                           index=hazus_plot.index)

# Add low/mid/high to naccs_plot
naccs_plot[['low', 'mid', 'high']] = pd.DataFrame(naccs_plot['params'].tolist(),
                                                  index=naccs_plot.index)

In [ ]:
# Plot our One Story With Basement ddfs
fig, ax = plt.subplots(figsize=(8, 6),
                       dpi=600)

# First, plot no uncertainty as a black line
hazus_nounc_plot.plot(x='depth_ft',
                      y='rel_dam',
                      ax=ax,
                      lw=2,
                      legend=False,
                      color='black')

# Second, plot naccs mid as a dashed blue line
naccs_plot.plot(x='depth_ft',
                y='mid',
                ax=ax,
                lw=1,
                ls='dashed',
                legend=False,
                color='#0077BB')

# Third, plot naccs low/high as a solid area
ax.fill_between(naccs_plot['depth_ft'],
                naccs_plot['low'],
                naccs_plot['high'],
                alpha=.5,
                lw=0,
                color='#0077BB')

# Fourth, plot the hazus uncertainty low/high
ax.fill_between(hazus_plot['depth_ft'],
                hazus_plot['low'],
                hazus_plot['high'],
                alpha=.5,
                lw=0,
                color='#EE7733')

# Limit axis from -9.1 to +10.1
ax.set_xlim([-9.1, 10])

# Style the axes
ax.set_xlabel('Depth of Water (Ft.) Relative to First Floor', size=14)
ax.set_ylabel('Percentage of Structure Value Compromised', size=14)
ax.yaxis.set_major_formatter(ticker.PercentFormatter(xmax=1))
ax.tick_params(labelsize=14)

# Add a custom legend in the top left
legend_elements = [Line2D([0], [0], color='black', lw=2, label='HAZUS DDF No Uncertainty (Benchmark)'),
                   Patch(facecolor='#EE7733', alpha=.5,
                         label='HAZUS DDF Uniform Bounds'),
                   Line2D([0], [0], color='#0077BB', lw=1, ls='dashed',
                          label='NACCS DDF Mode'),
                   Patch(facecolor='#0077BB', alpha=.5,
                         label='NACCS DDF Triangular Bounds'),
                   ]
ax.legend(handles=legend_elements,
          bbox_to_anchor=(.35, .83),
          loc='center',
          fontsize='large',
          frameon=False)

fig.savefig('/jumbo/keller-lab/projects/icom/nsi_unc/figures/ddf_ex_2SWB.png',
            bbox_inches='tight',
            dpi=600)

In [ ]:
# Get a figure for uncertainty in structure value
fd_id = 570878975
# The fdid above has structure value of 168546 w/o unc
# The median w/i unc is 158055, so close enough
ens_struct = ens_df[ens_df['fd_id'] == fd_id]

# Plot of the pdf of val for the example structure vs.
# single value from default method
fig, ax = plt.subplots(figsize=(8, 6),
                       dpi=600)
ens_struct['val_s'].plot(kind='hist', ax=ax, color='blue', bins=100, alpha=.2,
                         label='Structure Value\nacross ensemble')
ax.vlines(168546, 0, 260, color='red', label='NSI Value (Benchmark)')
ax.vlines(ens_struct['val_s'].mean(), 0, 260, color='blue', label='Mean across ensemble')
ax.set_xlabel('Structure Value ($)', size=14)
ax.set_ylabel('Frequency Across Ensemble Members', size=14)
ax.tick_params(labelsize=14)
ax.legend(loc='upper right')


# Save fig
fig.savefig('/jumbo/keller-lab/projects/icom/nsi_unc/figures/val_unc.png',
            bbox_inches='tight',
            dpi=600)

In [ ]:
# Get a figure for uncertainty in structure value
fd_id = 570878975
# The fdid above is 1SWB
ens_struct = ens_df[ens_df['fd_id'] == fd_id]

ens_struct['archetype'] = ens_struct['fnd_type'] + '_' + ens_struct['stories']
a_rename = {'B_1S': "Basement\n1 Story",
            'B_2S': 'Basement\n2 Stories',
            'C_1S': 'Crawl Space\n1 Story',
            'C_2S': 'Crawl Space\n2 Stories',
            'S_1S': "Slab\n1 Story",
            "S_2S": 'Slab\n2 Stories'}
ens_struct['atype'] = ens_struct['archetype'].map(a_rename)
counts = ens_struct.groupby(['atype']).size()
colors = ['#CC3311'] + ['#0077BB']*5

fig, ax = plt.subplots(figsize=(10, 6))

ax.barh(counts.index, counts.values,
        color=colors)

ax.tick_params(labelsize=12)

ax.set_xlabel('Frequency Across Ensemble Members', size=14)

# Add a custom legend in the top left
legend_elements = [Patch(facecolor='#CC3311',
                         label='NSI Default Value'),
                   ]
ax.legend(handles=legend_elements,
          bbox_to_anchor=(.82, .93),
          loc='center',
          fontsize='x-large',
          frameon=False)

fig.savefig('/jumbo/keller-lab/projects/icom/nsi_unc/figures/fnd_type.png',
            bbox_inches='tight',
            dpi=600)

In [ ]:
ens_struct['ffe'].describe()

In [ ]:
# Get a figure for uncertainty in first floor elevation
fd_id = 570878975
# The fdid above has ffe = 2.0
ens_struct = ens_df[ens_df['fd_id'] == fd_id]

# 
counts = ens_struct.groupby(['ffe']).size()
colors = ['#0077BB']*2 + ['#CC3311'] + ['#0077BB']*2

fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(counts.index, counts.values,
        color=colors)

ax.tick_params(labelsize=12)

ax.set_ylabel('Frequency Across Ensemble Members', size=14)
ax.set_xlabel('First Floor Elevation (Feet)', size=14)

# Add a custom legend in the top left
legend_elements = [Patch(facecolor='#CC3311',
                         label='NSI Default Value'),
                   ]
ax.legend(handles=legend_elements,
          bbox_to_anchor=(.82, .93),
          loc='center',
          fontsize='x-large',
          frameon=False)


# Save fig
fig.savefig('/jumbo/keller-lab/projects/icom/nsi_unc/figures/ffe_unc.png',
            bbox_inches='tight',
            dpi=600)